# Tutorial
This tutorial shows how to use the mllp package.

In [1]:
import torch
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

from mllp.models import MLLP
from mllp.utils import DBEncoder

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

/var/folders/x3/cd482yl51lg4jcy2wdj4q9sc0000gn/T/ipykernel_2671/2218189224.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Prepare data
We use the [breast cancer wisconsin dataset](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)) for example.  
X_df stores the feature values of all the instances.  
y_df stores the class labels of all the instances.  
f_df stores the feature names and feature types (continuous or discrete).

In [2]:
data = load_breast_cancer()

X_df = pd.DataFrame(data['data'], columns=data['feature_names'])
y_df = pd.DataFrame(data['target'], columns=['class'])
f_df = pd.DataFrame(zip(data['feature_names'], ['continuous'] * len(data.feature_names)))
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, train_size=0.8)

## Discrete and binarize data

For features in the data set are continuous (real numbers), we need to discrete them first.  
After data discretization, we use the one-hot encoding to encode all the features and the class.  
`DBEncoder` does all of this for us.

In [3]:
db_enc = DBEncoder(f_df, discrete=True)
db_enc.fit(X_train, y_train)
X_train, y_train = db_enc.transform(X_train, y_train)
X_test, y_test = db_enc.transform(X_test, y_test)

/Users/gonzalojaimovitch/Documents/projects/tfm/git/mllp/mllp/discretizer.py:71: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  c_cnt1[row[1]] += 1
/Users/gonzalojaimovitch/Documents/projects/tfm/git/mllp/mllp/discretizer.py:72: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if i + 1 < r and row[0] == df.iloc[i + 1][0]:
/Users/gonzalojaimovitch/Documents/projects/tfm/git/mllp/mllp/discretizer.py:100: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.i

`DBEncoder` uses the recursive minimal entropy partitioning algorithm for data discretization.  
The class member `me_discretizer` is the discretizer and we can get the partition boundaries by:

In [4]:
db_enc.me_discretizer.boundaries

defaultdict(list,
            {'mean radius': [13.16, 15.04, 17.85],
             'mean texture': [18.42],
             'mean perimeter': [85.24, 96.39, 114.6],
             'mean area': [530.6, 690.2, 992.1],
             'mean smoothness': [0.08853],
             'mean compactness': [0.1021, 0.1483],
             'mean concavity': [0.05192, 0.0905, 0.1374],
             'mean concave points': [0.02738, 0.05102, 0.07857],
             'mean symmetry': [0.1718, 0.2057],
             'mean fractal dimension': [],
             'radius error': [0.1935, 0.4202, 0.5619],
             'texture error': [],
             'perimeter error': [1.529, 2.759, 4.36],
             'area error': [18.51, 31.24, 53.65],
             'smoothness error': [],
             'compactness error': [0.0182],
             'concavity error': [0.01099, 0.02095],
             'concave points error': [0.009064, 0.01167],
             'symmetry error': [0.04197],
             'fractal dimension error': [0.002671],
    

## Set the MLLP
Set the network structure, device, rate of random binarization and whether use the NOT (~) operator.  
Run `MLLP?` for more information.

In [5]:
net_structure = [X_train.shape[-1], 32, y_train.shape[-1]]
# below is a more complex MLLP structure that can be used for a complex data set
# net_structure = [X_train.shape[-1], 128, 128, 64, y_train.shape[-1]]
net = MLLP(net_structure,
           device=device,
           random_binarization_rate=0.0,
           use_not=False)
net.to(device)

MLLP(
  (conj0): ConjunctionLayer(
    (randomly_binarize_layer): RandomBinarizationLayer()
  )
  (disj0): DisjunctionLayer(
    (randomly_binarize_layer): RandomBinarizationLayer()
  )
)

## Train the MLLP
Set the parameters for training and train the MLLP. The log is displayed during the training.  
Run `MLLP.train?` for more information.

In [6]:
training_log = net.train(
    X_train,
    y_train,
    lr=0.005,
    batch_size=16,
    epoch=100,
    lr_decay_rate=0.75,
    lr_decay_epoch=100,
    weight_decay=1e-7)

[INFO] - LR is set to 0.005
[INFO] - epoch: 0, loss: 7.055806383490562
[INFO] - ------------------------------------------------------------
[INFO] - On Training Set:
	Accuracy of MLLP Model: 0.5714285714285714
	Accuracy of CRS  Model: 0.42857142857142855
[INFO] - On Training Set:
	F1 Score of MLLP Model: 0.36363636363636365
	F1 Score of CRS  Model: 0.3
[INFO] - ------------------------------------------------------------
[INFO] - epoch: 1, loss: 3.5095775574445724
[INFO] - epoch: 2, loss: 2.16383795812726
[INFO] - epoch: 3, loss: 1.8785468265414238
[INFO] - epoch: 4, loss: 1.7090795412659645
[INFO] - epoch: 5, loss: 1.5704785380512476
[INFO] - ------------------------------------------------------------
[INFO] - On Training Set:
	Accuracy of MLLP Model: 1.0
	Accuracy of CRS  Model: 0.2857142857142857
[INFO] - On Training Set:
	F1 Score of MLLP Model: 1.0
	F1 Score of CRS  Model: 0.2222222222222222
[INFO] - ------------------------------------------------------------
[INFO] - epoch: 6,

## Test the trained MLLP and extracted CRS

In [7]:
acc, acc_b, f1, f1_b = net.test(X_test, y_test, need_transform=True)

print('Accuracy of MLLP Model: {}'
      '\nAccuracy of CRS  Model: {}'
      '\nF1 Score of MLLP Model: {}'
      '\nF1 Score of CRS  Model: {}'.format(acc, acc_b, f1, f1_b))

Accuracy of MLLP Model: 0.9385964912280702
Accuracy of CRS  Model: 0.9122807017543859
F1 Score of MLLP Model: 0.9383640997914575
F1 Score of CRS  Model: 0.9122536945812808


## Display the extracted CRS

In [8]:
net.concept_rule_set_print(X_fname=db_enc.X_fname, y_fname=db_enc.y_fname, eliminate_redundancy=True)

------------------------------------------------------------------------------------------
 class_0:
	       r1,0:	 [' mean radius_(12.32, 15.0]', ' mean perimeter_(81.09, 98.22]', ' mean smoothness_>0.08992', ' worst area_(694.4, 862.0]', ' worst smoothness_>0.136', ' worst compactness_>0.2809']
	       r1,4:	 [' mean radius_(12.32, 15.0]', ' mean texture_>19.6', ' mean perimeter_(81.09, 98.22]', ' mean smoothness_>0.08992', ' worst texture_>24.85', ' worst smoothness_>0.136']
	       r1,9:	 [' worst radius_(16.76, 18.22]', ' worst texture_>24.85', ' worst perimeter_(107.1, 120.3]', ' worst area_(862.0, 1032.0]', ' worst concavity_(0.2151, 0.3662]']
	      r1,11:	 [' worst perimeter_>120.3']
	      r1,13:	 [' mean symmetry_<=0.1714', ' worst radius_(16.76, 18.22]', ' worst perimeter_(107.1, 120.3]', ' worst area_(862.0, 1032.0]', ' worst concavity_(0.2151, 0.3662]']
	      r1,15:	 [' mean texture_>19.6', ' mean compactness_(0.05113, 0.1021]', ' worst radius_(14.98, 16.76]', ' worst te